In [ ]:
"""
Script to calculate minimum mask across all participants (i.e. mask only including voxels present in all partipants)
"""

import os
import numpy as np
import nilearn.image as image
import logging

# clear existing log handlers
logging.getLogger('').handlers = []

# setup logging
logfile_path = '/path/OBS_bayes_minMask_logfile.log'
logging.basicConfig(filename=logfile_path, level=logging.INFO, 
                    format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

# base path for the data
basepath = '/path/results_fmri'

# excluded participant IDs
excluded_participant_ids = [] # expects list of strings containing participant IDs that are to be excluded from the analysis

# list participant directories and filter
all_participant_dirs = os.listdir(basepath)
participant_ids = [pid for pid in all_participant_dirs if pid not in excluded_participant_ids]

# path and File Name Templates
file_template = os.path.join(basepath, 'sub-{0}', 'func', 'task-stress', 'sub-{0}_task-stress_feature-taskStress1_taskcontrast-stress_stat-effect_statmap.nii.gz')

# load images and calculate non-zero voxels
image_files = []
non_zero_voxels = []
for pid in participant_ids:
    img_path = file_template.format(pid.replace('sub-', ''))
    if 'sub-' in pid and os.path.exists(img_path):
        image_files.append(img_path)
        img_data = image.load_img(img_path).get_fdata()
        non_zero_voxels.append(np.sum(img_data != 0))

if not image_files:
    raise ValueError("No valid image files found for the included participants.")

mean_voxels = np.mean(non_zero_voxels)
std_voxels = np.std(non_zero_voxels)
threshold = 3  # define threshold for significant deviation 2sds away from the mean

# mask creation
first_img = image.load_img(image_files[0])
mask = np.ones(first_img.shape, dtype=bool)

# update mask and identify participants with significant deviation
participants_with_deviation = []
for img_file, voxels, pid in zip(image_files, non_zero_voxels, participant_ids):
    deviation = (mean_voxels - voxels) / std_voxels  # calculate deviation from the mean
    if deviation > threshold:  # check if deviation indicates significantly less voxels
        participants_with_deviation.append(pid)  # append participant ID only
    img = image.load_img(img_file)
    img_data = img.get_fdata()
    mask &= (img_data != 0)  # update mask with non-zero voxels
# convert boolean mask to binary mask
binary_mask = mask.astype(int)

# save mask
mask_img = image.new_img_like(first_img, binary_mask)
mask_filename = '/paths/minimum_mask.nii.gz'
mask_img.to_filename(mask_filename)
logging.info(f"Minimum mask saved to {mask_filename}")

# log participants with significant deviation
for pid in participants_with_deviation:
    logging.info(f"ID {pid} has significantly less non-zero voxels than average.")

# creating new mask without outliers  
outlier_participant_ids = [pid for pid in participants_with_deviation]

# filter image files to exclude outliers
non_outlier_image_files = [f for f, pid in zip(image_files, participant_ids) if pid not in outlier_participant_ids]

# initialize new mask with first non-outlier image
if non_outlier_image_files:
    first_non_outlier_img = image.load_img(non_outlier_image_files[0])
    non_outlier_mask = np.ones(first_non_outlier_img.shape, dtype=bool)

    # update mask to include only voxels present in all non-outlier participants
    for img_file in non_outlier_image_files:
        img = image.load_img(img_file)
        img_data = img.get_fdata()
        non_outlier_mask &= (img_data != 0)

    # convert boolean mask to binary mask
    non_outlier_binary_mask = non_outlier_mask.astype(np.int32)  # convert to int32 to avoid datatype issues

    # save new mask to file
    non_outlier_mask_img = image.new_img_like(first_non_outlier_img, non_outlier_binary_mask)
    non_outlier_mask_filename = '/path/minimum_mask_no_outliers.nii.gz'
    non_outlier_mask_img.to_filename(non_outlier_mask_filename)
    logging.info(f"Minimum mask excluding outliers saved to {non_outlier_mask_filename}")
else:
    logging.info("No valid non-outlier image files found. Cannot create a minimum mask excluding outliers.")
    
# final log
logging.info("Minimum mask script execution completed.")